In [ ]:
# # Imports
# !virtualenv myenv
# !conda create --name myenv
# !source myenv/bin/activate

# !conda activate myenv
!pip install py7zr

In [ ]:
!pip install ipywidgets

In [ ]:
import py7zr

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.stats import norm
import pandas as pd

%matplotlib inline
import IPython.display
from ipywidgets import interact, interactive, fixed

In [ ]:
%matplotlib inline
import IPython.display
from ipywidgets import interact, interactive, fixed

# Packages we're using
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage

import glob
import os
from tqdm import tqdm
# plt, plot, tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Packages we're using
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage
from tqdm import tqdm

In [ ]:
import numpy as np
import math
from PIL import Image
import time

In [ ]:
# Initializations

In [ ]:
FFT_LENGTH = 512
WINDOW_LENGTH = 512
WINDOW_STEP = int(WINDOW_LENGTH / 2)
magnitudeMin = float("inf")
magnitudeMax = float("-inf")
phaseMin = float("inf")
phaseMax = float("-inf")

In [ ]:
def amplifyMagnitudeByLog(d):
    return 188.301 * math.log10(d + 1)

def weakenAmplifiedMagnitude(d):
    return math.pow(10, d/188.301)-1

def generateLinearScale(magnitudePixels, phasePixels, 
                        magnitudeMin, magnitudeMax, phaseMin, phaseMax):
    height = magnitudePixels.shape[0]
    width = magnitudePixels.shape[1]
    magnitudeRange = magnitudeMax - magnitudeMin
    phaseRange = phaseMax - phaseMin
    rgbArray = np.zeros((height, width, 3), 'uint8')
    
    for w in range(width):
        for h in range(height):
            magnitudePixels[h,w] = (magnitudePixels[h,w] - magnitudeMin) / (magnitudeRange) * 255 * 2
            magnitudePixels[h,w] = amplifyMagnitudeByLog(magnitudePixels[h,w])
            phasePixels[h,w] = (phasePixels[h,w] - phaseMin) / (phaseRange) * 255
            red = 255 if magnitudePixels[h,w] > 255 else magnitudePixels[h,w]
            green = (magnitudePixels[h,w] - 255) if magnitudePixels[h,w] > 255 else 0
            blue = phasePixels[h,w]
            rgbArray[h,w,0] = int(red)
            rgbArray[h,w,1] = int(green)
            rgbArray[h,w,2] = int(blue)
    return rgbArray

def recoverLinearScale(rgbArray, magnitudeMin, magnitudeMax, 
                       phaseMin, phaseMax):
    width = rgbArray.shape[1]
    height = rgbArray.shape[0]
    print(phaseMax,phaseMin)
    magnitudeVals = rgbArray[:,:,0].astype(float) + rgbArray[:,:,1].astype(float)
    phaseVals = rgbArray[:,:,2].astype(float)
    phaseRange = phaseMax - phaseMin
    magnitudeRange = magnitudeMax - magnitudeMin
     
#     print(magnitudeVals)
#     print(phaseVals)
#     print(phaseRange)
#     print(magnitudeRange)
    
    for w in range(width):
        for h in range(height):
            phaseVals[h,w] = (phaseVals[h,w] / 255 * phaseRange) + phaseMin
            magnitudeVals[h,w] = weakenAmplifiedMagnitude(magnitudeVals[h,w])
            magnitudeVals[h,w] = (magnitudeVals[h,w] / (255*2) * magnitudeRange) + magnitudeMin
    return magnitudeVals, phaseVals

In [ ]:
def generateSpectrogramForWave(signal):
    start_time = time.time() #calculates current time
    global magnitudeMin
    global magnitudeMax
    global phaseMin
    global phaseMax
    buffer = np.zeros(int(signal.size + WINDOW_STEP - (signal.size % WINDOW_STEP)))
    buffer[0:len(signal)] = signal
    height = int(FFT_LENGTH / 2 + 1)
    width = int(len(buffer) / (WINDOW_STEP) - 1)
    magnitudePixels = np.zeros((height, width))
    phasePixels = np.zeros((height, width))

    for w in range(width):
        buff = np.zeros(FFT_LENGTH)
        stepBuff = buffer[w*WINDOW_STEP:w*WINDOW_STEP + WINDOW_LENGTH]
        # apply hanning window
        stepBuff = stepBuff * np.hanning(WINDOW_LENGTH)
        buff[0:len(stepBuff)] = stepBuff
        #buff now contains windowed signal with step length and padded with zeroes to the end
        fft = np.fft.rfft(buff)
        for h in range(len(fft)):
            #print(buff.shape)
            #return
            magnitude = math.sqrt(fft[h].real**2 + fft[h].imag**2)
            if magnitude > magnitudeMax:
                magnitudeMax = magnitude 
            if magnitude < magnitudeMin:
                magnitudeMin = magnitude 

            phase = math.atan2(fft[h].imag, fft[h].real)
            if phase > phaseMax:
                phaseMax = phase
            if phase < phaseMin:
                phaseMin = phase
            magnitudePixels[height-h-1,w] = magnitude
            phasePixels[height-h-1,w] = phase
    rgbArray = generateLinearScale(magnitudePixels, phasePixels,
                                  magnitudeMin, magnitudeMax, phaseMin, phaseMax)
    
    
    elapsed_time = time.time() - start_time
    print('%.2f' % elapsed_time, 's', sep='')
    img = Image.fromarray(rgbArray, 'RGB')
    return img

print("phaseMax {} phaseMin {} magnitudeMax {} magnitudeMin {}".format(phaseMax,phaseMin,magnitudeMax,magnitudeMin))

In [ ]:
print("phaseMax {} phaseMin {} magnitudeMax {} magnitudeMin {}".format(phaseMax,phaseMin,magnitudeMax,magnitudeMin))

In [ ]:
# # !pip install py7zr
# print(py7zr.__version__)
# # !python3 -m venv /myenv
# # !source /myenv/bin/activate

In [ ]:
!pip install py7zr

In [ ]:
import py7zr

def extract_7z(file_path, output_path):
    with py7zr.SevenZipFile(file_path, mode='r') as z:
        z.extractall(output_path)

# Usage example
# file_path = '/kaggle/input/libri-speech-noise-dataset/test.7z'
output_path = '/kaggle/working/'


extract_7z("/kaggle/input/libri-speech-noise-dataset/y_train.7z", output_path)
extract_7z("/kaggle/input/libri-speech-noise-dataset/train.7z", output_path)
extract_7z("/kaggle/input/libri-speech-noise-dataset/y_test.7z", output_path)

In [ ]:
path_train = "/kaggle/working/train"
path_test = '/kaggle/working/test'
path_ytrain = '/kaggle/working/y_train'
path_ytest = '/kaggle/working/y_test'

In [ ]:
path_WAVA = './WAVA'
path_WAVA = './WAVB'

In [ ]:
import random
import shutil

random.seed(42)

In [ ]:
# Create 1 sec wav files and store it in *.npy format

In [ ]:
for filename in tqdm(glob.glob(os.path.join(path_train, '*.wav'))):
    rate, data = wavfile.read(filename)
    if len(data.shape) >= 2 and data.size > 0:
        if data.shape[-1] > 1:
            data = data.mean(axis=-1)
        else:
            data = np.reshape(data, data.shape[:-1])
        
    if np.shape(data)[0] / float(rate) > 1:
        data = data[0 : rate * 1]
            
    img = generateSpectrogramForWave(data)
#     img.save(filename[:-4]+'.png')
    np.save(filename[:-4]+'.npy', img) # save

In [ ]:
print("phaseMax {} phaseMin {} magnitudeMax {} magnitudeMin {}".format(phaseMax,phaseMin,magnitudeMax,magnitudeMin))

In [ ]:
for filename in tqdm(glob.glob(os.path.join(path_test, '*.wav'))):
    rate, data = wavfile.read(filename)
    if len(data.shape) >= 2 and data.size > 0:
        if data.shape[-1] > 1:
            data = data.mean(axis=-1)
        else:
            data = np.reshape(data, data.shape[:-1])
        
    if np.shape(data)[0] / float(rate) > 1:
        data = data[0 : rate * 1]
            
    img = generateSpectrogramForWave(data)
    np.save(filename[:-4]+'.npy', img) # save

In [ ]:
print("phaseMax {} phaseMin {} magnitudeMax {} magnitudeMin {}".format(phaseMax,phaseMin,magnitudeMax,magnitudeMin))

In [ ]:
for filename in tqdm(glob.glob(os.path.join(path_ytrain, '*.wav'))):
    rate, data = wavfile.read(filename)
    if len(data.shape) >= 2 and data.size > 0:
        if data.shape[-1] > 1:
            data = data.mean(axis=-1)
        else:
            data = np.reshape(data, data.shape[:-1])
        
    if np.shape(data)[0] / float(rate) > 1:
        data = data[0 : rate * 1]
            
    img = generateSpectrogramForWave(data)
    np.save(filename[:-4]+'.npy', img) # save

In [ ]:
print("phaseMax {} phaseMin {} magnitudeMax {} magnitudeMin {}".format(phaseMax,phaseMin,magnitudeMax,magnitudeMin))

In [ ]:
for filename in tqdm(glob.glob(os.path.join(path_ytest, '*.wav'))):
    rate, data = wavfile.read(filename)
    if len(data.shape) >= 2 and data.size > 0:
        if data.shape[-1] > 1:
            data = data.mean(axis=-1)
        else:
            data = np.reshape(data, data.shape[:-1])
        
    if np.shape(data)[0] / float(rate) > 1:
        data = data[0 : rate * 1]
            
    img = generateSpectrogramForWave(data)
    np.save(filename[:-4]+'.npy', img) # save

In [ ]:
print("phaseMax {} phaseMin {} magnitudeMax {} magnitudeMin {}".format(phaseMax,phaseMin,magnitudeMax,magnitudeMin))

In [ ]:
path_mat_train = '/kaggle/input/libri-speech-noise-dataset/LibriNoise_Train_Test_NPY/mat_train'
path_mat_test = '/kaggle/input/libri-speech-noise-dataset/LibriNoise_Train_Test_NPY/mat_test'
path_mat_ytrain = '/kaggle/input/libri-speech-noise-dataset/LibriNoise_Train_Test_NPY/mat_ytrain'
path_mat_ytest = '/kaggle/input/libri-speech-noise-dataset/LibriNoise_Train_Test_NPY/mat_ytest'

In [ ]:
#copies files from path_ to path_mat_ and deletes files from path_
for filename in glob.glob(os.path.join(path_train, '*.npy')):
    shutil.copy(filename, path_mat_train)
    os.remove(filename)
for filename in glob.glob(os.path.join(path_ytrain, '*.npy')):
    shutil.copy(filename, path_mat_ytrain)
    os.remove(filename) 

    
for filename in glob.glob(os.path.join(path_test, '*.npy')):
    shutil.copy(filename, path_mat_test)
    os.remove(filename)    
for filename in glob.glob(os.path.join(path_ytest, '*.npy')):
    shutil.copy(filename, path_mat_ytest)
    os.remove(filename)

In [ ]:
# Use below initialization values for recovery

In [ ]:
phaseMax = 3.141592653589793 
phaseMin = -3.141592653589793
magnitudeMax = 2211683.973249525
magnitudeMin = 0.0

rate = 16000

In [ ]:
def recoverSignalFromSpectrogram(numpyarray):
#     img = Image.open(filePath)
    data = np.array( numpyarray, dtype='uint8' )
    width = data.shape[1]
    height = data.shape[0]

    magnitudeVals, phaseVals  = recoverLinearScale(data, magnitudeMin, magnitudeMax, phaseMin, phaseMax)
   
        
    recovered = np.zeros(WINDOW_LENGTH * width // 2 + WINDOW_STEP, dtype=np.int16)
    recovered = np.array(recovered,dtype=np.int16)
    
    for w in range(width):
        toInverse = np.zeros(height, dtype=np.complex_)
        for h in range(height):
            magnitude = magnitudeVals[height-h-1,w]
            phase = phaseVals[height-h-1,w]
            toInverse[h] = magnitude * math.cos(phase) + (1j * magnitude * math.sin(phase))
        signal = np.fft.irfft(toInverse)
        recovered[w*WINDOW_STEP:w*WINDOW_STEP + WINDOW_LENGTH] += signal[:WINDOW_LENGTH].astype(np.int16)
    return recovered

In [ ]:
# Test the recovery - convert npy to wav

In [ ]:
# filename = path_mat_test+'/'+'19-198-0034.npy'
# new_num_arr = np.load(filename)
# new_num_arr.shape
# data = recoverSignalFromSpectrogram(new_num_arr)
# scipy.io.wavfile.write('./'+filename[:-4]+'.wav', rate, data)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.stats import norm
import pandas as pd

%matplotlib inline
import IPython.display
from ipywidgets import interact, interactive, fixed
%matplotlib inline
import IPython.display
from ipywidgets import interact, interactive, fixed

# Packages we're using
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage

import glob
import os
from tqdm import tqdm
# plt, plot, tqdm
import warnings
warnings.filterwarnings('ignore')
# Packages we're using
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import numpy as np
import math
from PIL import Image
import time
path_train = '/kaggle/working/train'
path_test = '/kaggle/working/test'
path_ytrain = '/kaggle/working/y_train'
path_ytest = '/kaggle/working/y_test'

path_mat_train = '/kaggle/input/libri-speech-noise-dataset/LibriNoise_Train_Test_NPY/mat_train'
path_mat_test = '/kaggle/input/libri-speech-noise-dataset/LibriNoise_Train_Test_NPY/mat_test'
path_mat_ytrain = '/kaggle/input/libri-speech-noise-dataset/LibriNoise_Train_Test_NPY/mat_ytrain'
path_mat_ytest = '/kaggle/input/libri-speech-noise-dataset/LibriNoise_Train_Test_NPY/mat_ytest'
ROW = 257
COL = 62
INPUT_DIM = (ROW,COL,3)
INPUT_DIM[:2]

In [ ]:
import keras
from matplotlib import pyplot as plt
import numpy as np
import gzip
%matplotlib inline
from keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D,concatenate
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.models import Model
from keras.optimizers import RMSprop
FFT_LENGTH = 512
WINDOW_LENGTH = 512
WINDOW_STEP = int(WINDOW_LENGTH / 2)

phaseMax = 3.141592653589793 
phaseMin = -3.141592653589793
magnitudeMax = 2211683.973249525
magnitudeMin = 0.0
def amplifyMagnitudeByLog(d):
    return 188.301 * math.log10(d + 1)

def weakenAmplifiedMagnitude(d):
    return math.pow(10, d/188.301)-1

def generateLinearScale(magnitudePixels, phasePixels, 
                        magnitudeMin, magnitudeMax, phaseMin, phaseMax):
    height = magnitudePixels.shape[0]
    width = magnitudePixels.shape[1]
    magnitudeRange = magnitudeMax - magnitudeMin
    phaseRange = phaseMax - phaseMin
    rgbArray = np.zeros((height, width, 3), 'uint8')
    
    for w in range(width):
        for h in range(height):
            magnitudePixels[h,w] = (magnitudePixels[h,w] - magnitudeMin) / (magnitudeRange) * 255 * 2
            magnitudePixels[h,w] = amplifyMagnitudeByLog(magnitudePixels[h,w])
            phasePixels[h,w] = (phasePixels[h,w] - phaseMin) / (phaseRange) * 255
            red = 255 if magnitudePixels[h,w] > 255 else magnitudePixels[h,w]
            green = (magnitudePixels[h,w] - 255) if magnitudePixels[h,w] > 255 else 0
            blue = phasePixels[h,w]
            rgbArray[h,w,0] = int(red)
            rgbArray[h,w,1] = int(green)
            rgbArray[h,w,2] = int(blue)
    return rgbArray

def recoverLinearScale(rgbArray, magnitudeMin, magnitudeMax, 
                       phaseMin, phaseMax):
    width = rgbArray.shape[1]
    height = rgbArray.shape[0]
#     print(phaseMax,phaseMin)
    magnitudeVals = rgbArray[:,:,0].astype(float) + rgbArray[:,:,1].astype(float)
    phaseVals = rgbArray[:,:,2].astype(float)
    phaseRange = phaseMax - phaseMin
    magnitudeRange = magnitudeMax - magnitudeMin
    
#     print(magnitudeVals)
#     print(phaseVals)
#     print(phaseRange)
#     print(magnitudeRange)
    
    for w in range(width):
        for h in range(height):
            phaseVals[h,w] = (phaseVals[h,w] / 255 * phaseRange) + phaseMin
            magnitudeVals[h,w] = weakenAmplifiedMagnitude(magnitudeVals[h,w])
            magnitudeVals[h,w] = (magnitudeVals[h,w] / (255*2) * magnitudeRange) + magnitudeMin
    return magnitudeVals, phaseVals

In [ ]:
def recoverSignalFromSpectrogram(numpyarray):
#     img = Image.open(filePath)
    data = np.array( numpyarray, dtype='uint8' )
    width = data.shape[1]
    height = data.shape[0]
    
#     print(phaseMax,phaseMin)

    magnitudeVals, phaseVals \
    = recoverLinearScale(data, magnitudeMin, magnitudeMax, phaseMin, phaseMax)
    
#     print(magnitudeVals,phaseVals)
    
    recovered = np.zeros(WINDOW_LENGTH * width // 2 + WINDOW_STEP, dtype=np.int16)
    recovered = np.array(recovered,dtype=np.int16)
    
    for w in range(width):
        toInverse = np.zeros(height, dtype=np.complex_)
        for h in range(height):
            magnitude = magnitudeVals[height-h-1,w]
            phase = phaseVals[height-h-1,w]
            toInverse[h] = magnitude * math.cos(phase) + (1j * magnitude * math.sin(phase))
        signal = np.fft.irfft(toInverse)
        recovered[w*WINDOW_STEP:w*WINDOW_STEP + WINDOW_LENGTH] += signal[:WINDOW_LENGTH].astype(np.int16)
    return recovered

In [ ]:
nb_train_samples = 7000 # change here
nb_test_samples = 105

train_batch_size = [5,4,3,2]
test_batch_size = [5,4,3,2]

epochs = 30
inChannel = 3
x, y = INPUT_DIM[:2]
input_img = Input(shape = (x, y, inChannel))

In [ ]:
glob.glob(os.path.join(path, '*.npy'))

In [ ]:
train_image = []
path = path_mat_train
for filename in tqdm(glob.glob(os.path.join(path, '*.npy'))):
    train_image.append(os.path.join(filename))
    
ytrain_image = []
path = path_mat_ytrain
for filename in tqdm(glob.glob(os.path.join(path, '*.npy'))):
    ytrain_image.append(os.path.join(filename))
    
test_image = []
path = path_mat_test
for filename in tqdm(glob.glob(os.path.join(path, '*.npy'))):
    test_image.append(os.path.join(filename))
    
ytest_image = []
path = path_mat_ytest
for filename in tqdm(glob.glob(os.path.join(path, '*.npy'))):
    ytest_image.append(os.path.join(filename))

In [ ]:
def data_gen_train(train_batch_size):
    while True:
        for start in range(0,nb_train_samples,train_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + train_batch_size, nb_train_samples)
            for img_path in range(start, end):
                img_train = np.load(train_image[img_path])
                img_train = img_train/255
                x_batch.append(img_train)
                img_ytrain = np.load(ytrain_image[img_path])
                img_ytrain = img_ytrain/255
                y_batch.append(img_ytrain)
            yield (np.array(x_batch), np.array(y_batch)) 
def data_gen_test(test_batch_size):
    while True:
        for start in range(0,nb_test_samples,test_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + test_batch_size, nb_test_samples)
            for img_path in range(start, end):
                img_test = np.load(test_image[img_path])
                img_test = img_test/255
                x_batch.append(img_test)
                img_ytest= np.load(ytest_image[img_path])
                img_ytest = img_ytest/255
                y_batch.append(img_ytest)
            yield (np.array(x_batch), np.array(y_batch))

In [ ]:
def autoencoder(input_img):
    #encoder
    conv1 = Conv2D(32, (2, 2), strides=(1, 1), activation='relu', padding='same')(input_img)
    pool1 = MaxPooling2D(pool_size=(1, 1))(conv1)
    pool1 = Dropout(0.5)(pool1)
    conv2 = Conv2D(64, (2, 2), strides=(1, 1), activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D(pool_size=(1, 1))(conv2)
    pool2 = Dropout(0.5)(pool2)
    conv3 = Conv2D(128, (2, 2), strides=(1, 1), activation='relu', padding='same')(pool2)
    pool3 = MaxPooling2D(pool_size=(1, 1))(conv3)
    pool3 = Dropout(0.5)(pool3)
    conv4 = Conv2D(256, (2, 2), strides=(1, 1), activation='relu', padding='same')(pool3)
    
    #decoder
    conv5 = Conv2D(256, (2, 2),strides=(1, 1), activation='relu', padding='same')(conv4)
    skip1 = concatenate([conv5, conv4], name='skip1')
    up1 = UpSampling2D((1,1))(skip1)
    up1 = Dropout(0.5)(up1)
    conv6 = Conv2D(128, (2, 2),strides=(1, 1), activation='relu', padding='same')(conv5)
    skip2 = concatenate([conv6, conv3], name='skip2')
    up2 = UpSampling2D((1,1))(skip2)
    up2 = Dropout(0.5)(up2)
    conv7 = Conv2D(64, (2, 2),strides=(1, 1), activation='relu', padding='same')(up2)
    skip3 = concatenate([conv7, conv2], name='skip3')
    up3 = UpSampling2D((1,1))(skip3)
    up3 = Dropout(0.5)(up3)
    decoded = Conv2D(3, (2, 2),strides=(1, 1), activation='sigmoid', padding='same')(up3)
    return decoded
from keras.optimizers import RMSprop, Adam
opt = RMSprop( learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False,name='RMSprop')
autoencoder = Model(input_img, autoencoder(input_img))
autoencoder.compile(loss='mse', optimizer = opt, metrics=['accuracy'])
autoencoder.summary()

In [ ]:
import keras.layers as layers
import keras.models as models
from keras.initializers import orthogonal


def Conv2DLayer(x, filters, kernel, strides, padding, block_id, kernel_init=orthogonal()):
    prefix = f'block_{block_id}_'
    x = layers.Conv2D(filters, kernel_size=kernel, strides=strides, padding=padding,
                      kernel_initializer=kernel_init, name=prefix+'conv')(x)
    x = layers.LeakyReLU(name=prefix+'lrelu')(x)
    x = layers.Dropout(0.2, name=prefix+'drop')((x))
    x = layers.BatchNormalization(name=prefix+'conv_bn')(x)
    return x

def Transpose_Conv2D(x, filters, kernel, strides, padding, block_id, kernel_init=orthogonal()):
    prefix = f'block_{block_id}_'
    x = layers.Conv2DTranspose(filters, kernel_size=kernel, strides=strides, padding=padding,
                               kernel_initializer=kernel_init, name=prefix+'de-conv')(x)
    x = layers.LeakyReLU(name=prefix+'lrelu')(x)
    x = layers.Dropout(0.2, name=prefix+'drop')((x))
    x = layers.BatchNormalization(name=prefix+'conv_bn')(x)
    return x



def AutoEncdoer(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    # 256 x 256
    conv1 = Conv2DLayer(inputs, 64, 3, strides=1, padding='same', block_id=1)
    conv2 = Conv2DLayer(conv1, 64, 3, strides=1, padding='same', block_id=2)
    
    # 128 x 128
    conv3 = Conv2DLayer(conv2, 128, 5, strides=1, padding='same', block_id=3)
    
    # 64 x 64
    conv4 = Conv2DLayer(conv3, 128, 3, strides=1, padding='same', block_id=4)
    conv5 = Conv2DLayer(conv4, 256, 5, strides=1, padding='same', block_id=5)
    
    # 32 x 32
    conv6 = Conv2DLayer(conv5, 512, 3, strides=1, padding='same', block_id=6)
    
    # 16 x 16
    deconv1 = Transpose_Conv2D(conv6, 512, 3, strides=1, padding='same', block_id=7)
    
    # 32 x 32
    skip1 = layers.concatenate([deconv1, conv5], name='skip1')
    conv7 = Conv2DLayer(skip1, 256, 3, strides=1, padding='same', block_id=8)
    deconv2 = Transpose_Conv2D(conv7, 128, 3, strides=1, padding='same', block_id=9)
    
    # 64 x 64
    skip2 = layers.concatenate([deconv2, conv3], name='skip2')
    conv8 = Conv2DLayer(skip2, 128, 5, strides=1, padding='same', block_id=10)
    deconv3 = Transpose_Conv2D(conv8, 64, 3, strides=1, padding='same', block_id=11)
    
    # 128 x 128
    skip3 = layers.concatenate([deconv3, conv2], name='skip3')
    conv9 = Conv2DLayer(skip3, 64, 5, strides=1, padding='same', block_id=12)
    deconv4 = Transpose_Conv2D(conv9, 64, 3, strides=1, padding='same', block_id=13)
    
    # 256 x 256
    skip3 = layers.concatenate([deconv4, conv1])
    conv10 = layers.Conv2D(3, 3, strides=1, padding='same', activation='sigmoid',
                       kernel_initializer=orthogonal(), name='final_conv')(skip3)

    
    return models.Model(inputs=inputs, outputs=conv10)

In [ ]:
# import tensorflow as tf
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
ROW = 257
COL = 62

In [ ]:
from keras.optimizers import SGD, Adam, RMSprop 
# opt = RMSprop( learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False,name='RMSprop')
# opt = Adam(lr=0.001)
opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# with tpu_strategy.scope():
def AutoEncdoer(input_shape):
    inputs = layers.Input(shape=input_shape)

    # 256 x 256
    conv1 = Conv2DLayer(inputs, 64, 3, strides=1, padding='same', block_id=1)
    conv2 = Conv2DLayer(conv1, 64, 3, strides=1, padding='same', block_id=2)

    # 128 x 128
    conv3 = Conv2DLayer(conv2, 128, 5, strides=1, padding='same', block_id=3)

    # 64 x 64
    conv4 = Conv2DLayer(conv3, 128, 3, strides=1, padding='same', block_id=4)
    conv5 = Conv2DLayer(conv4, 256, 5, strides=1, padding='same', block_id=5)

    # 32 x 32
    conv6 = Conv2DLayer(conv5, 512, 3, strides=1, padding='same', block_id=6)

    # 16 x 16
    deconv1 = Transpose_Conv2D(conv6, 512, 3, strides=1, padding='same', block_id=7)

    # 32 x 32
    skip1 = layers.concatenate([deconv1, conv5], name='skip1')
    conv7 = Conv2DLayer(skip1, 256, 3, strides=1, padding='same', block_id=8)
    deconv2 = Transpose_Conv2D(conv7, 128, 3, strides=1, padding='same', block_id=9)

    # 64 x 64
    skip2 = layers.concatenate([deconv2, conv3], name='skip2')
    conv8 = Conv2DLayer(skip2, 128, 5, strides=1, padding='same', block_id=10)
    deconv3 = Transpose_Conv2D(conv8, 64, 3, strides=1, padding='same', block_id=11)

    # 128 x 128
    skip3 = layers.concatenate([deconv3, conv2], name='skip3')
    conv9 = Conv2DLayer(skip3, 64, 5, strides=1, padding='same', block_id=12)
    deconv4 = Transpose_Conv2D(conv9, 64, 3, strides=1, padding='same', block_id=13)

    # 256 x 256
    skip3 = layers.concatenate([deconv4, conv1])
    conv10 = layers.Conv2D(3, 3, strides=1, padding='same', activation='sigmoid',
                       kernel_initializer=orthogonal(), name='final_conv')(skip3)


    return models.Model(inputs=inputs, outputs=conv10)
autoencoder = AutoEncdoer((ROW,COL, 3))
autoencoder.compile(optimizer=opt, loss=['mae','mse'], metrics=['mse','accuracy'])

In [ ]:
# import tensorflow as tf
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
import os
import random

import tensorflow as tf
from tensorflow import keras


from tensorflow.python.keras.saving import saving_utils as _saving_utils
from tensorflow.python.framework import convert_to_constants as convert_variables_to_constants_v2

print(tf.version.VERSION)

In [ ]:
PERIOD = 5

# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "modelcheckpoints/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 10 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=PERIOD)

In [ ]:
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, autoencoder_train
                     ,epoch, logs=None):
        rate = 16000
        for j in range(5):
            r_num = random.randint(0, nb_test_samples-1)
            test_file = test_image[r_num]
            img_test = np.load(test_file)
            img_test = img_test/255
            img_test = img_test.reshape(-1, ROW,COL,3)
            decoded_imgs = autoencoder.predict(img_test) #predict
            decoded_imgs = decoded_imgs.reshape(ROW,COL,3)
            decoded_imgs = decoded_imgs*255
            decoded_imgs = decoded_imgs.astype(np.int16)
            data = recoverSignalFromSpectrogram(decoded_imgs)
            scipy.io.wavfile.write('./'+"predict_{}".format(j)+'.wav', rate, data)
        autoencoder.save('model_sgd.h5')
from matplotlib import pyplot

In [ ]:
for i in range(len(train_batch_size)):
    autoencoder_train = autoencoder.fit(data_gen_train(train_batch_size[i]),
                                    epochs= epochs,
                                    verbose=1,
                                    steps_per_epoch= nb_train_samples // train_batch_size[i],
                                    validation_data= data_gen_test(test_batch_size[i]),
                                    validation_steps=nb_test_samples // test_batch_size[i],
                                    callbacks=[cp_callback,CustomCallback()])
#     pyplot.subplot(211)
#     pyplot.title('Loss')
#     pyplot.plot(autoencoder_train.history['loss'], label='train')
#     pyplot.plot(autoencoder_train.history['val_loss'], label='test')
#     pyplot.legend()
#     pyplot.show()
#     pyplot.subplot(212)
#     pyplot.title('Mean Squared Error')
#     pyplot.plot(autoencoder_train.history['mean_squared_error'], label='train')
#     pyplot.plot(autoencoder_train.history['val_mean_squared_error'], label='test')
#     pyplot.legend()
#     pyplot.show()
#     # list all data in history
#     print(autoencoder_train.history.keys())
#     # summarize history for accuracy
#     plt.plot(autoencoder_train.history['accuracy'])
#     plt.plot(autoencoder_train.history['val_accuracy'])
#     plt.title('model accuracy')
#     plt.ylabel('accuracy')
#     plt.xlabel('epoch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()

In [ ]:
autoencoder.save("model_sgd")

In [ ]:
save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
model.save('./kaggle/working/model_test_run', options=save_locally) # saving in Tensorflow's "SavedModel" format

In [ ]:
autoencoder.save_weights(checkpoint_path.format(epoch=0))
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest
# 'modelcheckpoints\\cp-0000.ckpt'
# checkpoint_dir
# 'modelcheckpoints'
# # Load the previously saved weights
# autoencoder.load_weights(latest)

In [ ]:
from pydub import AudioSegment
from pydub.utils import make_chunks

In [ ]:
import numpy as np
import math
from PIL import Image
import time

# Packages we're using
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage
from tqdm import tqdm

%matplotlib inline
import IPython.display
from ipywidgets import interact, interactive, fixed

# Packages we're using
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage

import glob
import os
from tqdm import tqdm
# plt, plot, tqdm
import warnings
warnings.filterwarnings('ignore')
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
FFT_LENGTH = 512
WINDOW_LENGTH = 512
WINDOW_STEP = int(WINDOW_LENGTH / 2)
phaseMax = 3.141592653589793 
phaseMin = -3.141592653589793
magnitudeMax = 5432292.907520762 #3212111.6378762764 # 5432292.907520762
magnitudeMin = 0.0
def amplifyMagnitudeByLog(d):
    return 188.301 * math.log10(d + 1)

def weakenAmplifiedMagnitude(d):
    return math.pow(10, d/188.301)-1

def generateLinearScale(magnitudePixels, phasePixels, 
                        magnitudeMin, magnitudeMax, phaseMin, phaseMax):
    height = magnitudePixels.shape[0]
    width = magnitudePixels.shape[1]
    magnitudeRange = magnitudeMax - magnitudeMin
    phaseRange = phaseMax - phaseMin
    rgbArray = np.zeros((height, width, 3), 'uint8')
    
    for w in range(width):
        for h in range(height):
            magnitudePixels[h,w] = (magnitudePixels[h,w] - magnitudeMin) / (magnitudeRange) * 255 * 2
            magnitudePixels[h,w] = amplifyMagnitudeByLog(magnitudePixels[h,w])
            phasePixels[h,w] = (phasePixels[h,w] - phaseMin) / (phaseRange) * 255
            red = 255 if magnitudePixels[h,w] > 255 else magnitudePixels[h,w]
            green = (magnitudePixels[h,w] - 255) if magnitudePixels[h,w] > 255 else 0
            blue = phasePixels[h,w]
            rgbArray[h,w,0] = int(red)
            rgbArray[h,w,1] = int(green)
            rgbArray[h,w,2] = int(blue)
    return rgbArray

def recoverLinearScale(rgbArray, magnitudeMin, magnitudeMax, 
                       phaseMin, phaseMax):
    width = rgbArray.shape[1]
    height = rgbArray.shape[0]
    magnitudeVals = rgbArray[:,:,0].astype(float) + rgbArray[:,:,1].astype(float)
    phaseVals = rgbArray[:,:,2].astype(float)
    phaseRange = phaseMax - phaseMin
    magnitudeRange = magnitudeMax - magnitudeMin
    for w in range(width):
        for h in range(height):
            phaseVals[h,w] = (phaseVals[h,w] / 255 * phaseRange) + phaseMin
            magnitudeVals[h,w] = weakenAmplifiedMagnitude(magnitudeVals[h,w])
            magnitudeVals[h,w] = (magnitudeVals[h,w] / (255*2) * magnitudeRange) + magnitudeMin
    return magnitudeVals, phaseVals
def generateSpectrogramForWave(signal):
    start_time = time.time()
    global magnitudeMin
    global magnitudeMax
    global phaseMin
    global phaseMax
    buffer = np.zeros(int(signal.size + WINDOW_STEP - (signal.size % WINDOW_STEP)))
    buffer[0:len(signal)] = signal
    height = int(FFT_LENGTH / 2 + 1)
    width = int(len(buffer) / (WINDOW_STEP) - 1)
    magnitudePixels = np.zeros((height, width))
    phasePixels = np.zeros((height, width))

    for w in range(width):
        buff = np.zeros(FFT_LENGTH)
        stepBuff = buffer[w*WINDOW_STEP:w*WINDOW_STEP + WINDOW_LENGTH]
        # apply hanning window
        stepBuff = stepBuff * np.hanning(WINDOW_LENGTH)
        buff[0:len(stepBuff)] = stepBuff
        #buff now contains windowed signal with step length and padded with zeroes to the end
        fft = np.fft.rfft(buff)
        for h in range(len(fft)):
            #print(buff.shape)
            #return
            magnitude = math.sqrt(fft[h].real**2 + fft[h].imag**2)
            if magnitude > magnitudeMax:
                magnitudeMax = magnitude 
            if magnitude < magnitudeMin:
                magnitudeMin = magnitude 

            phase = math.atan2(fft[h].imag, fft[h].real)
            if phase > phaseMax:
                phaseMax = phase
            if phase < phaseMin:
                phaseMin = phase
            magnitudePixels[height-h-1,w] = magnitude
            phasePixels[height-h-1,w] = phase
    rgbArray = generateLinearScale(magnitudePixels, phasePixels,
                                  magnitudeMin, magnitudeMax, phaseMin, phaseMax)
    
    
    elapsed_time = time.time() - start_time
    print('%.2f' % elapsed_time, 's', sep='')
    img = Image.fromarray(rgbArray, 'RGB')
    return img
def recoverSignalFromSpectrogram(numpyarray):
#     img = Image.open(filePath)
    data = np.array( numpyarray, dtype='uint8' )
    width = data.shape[1]
    height = data.shape[0]

    magnitudeVals, phaseVals \
    = recoverLinearScale(data, magnitudeMin, magnitudeMax, phaseMin, phaseMax)
    
    recovered = np.zeros(WINDOW_LENGTH * width // 2 + WINDOW_STEP, dtype=np.int16)
    recovered = np.array(recovered,dtype=np.int16)
    
    for w in range(width):
        toInverse = np.zeros(height, dtype=np.complex_)
        for h in range(height):
            magnitude = magnitudeVals[height-h-1,w]
            phase = phaseVals[height-h-1,w]
            toInverse[h] = magnitude * math.cos(phase) + (1j * magnitude * math.sin(phase))
        signal = np.fft.irfft(toInverse)
        recovered[w*WINDOW_STEP:w*WINDOW_STEP + WINDOW_LENGTH] += signal[:WINDOW_LENGTH].astype(np.int16)
    return recovered
rate = 16000
print("phaseMax {} phaseMin {} magnitudeMax {} magnitudeMin {}".format(phaseMax,phaseMin,magnitudeMax,magnitudeMin))

In [ ]:
path_Voice = '/kaggle/working/noisesB/samples'
 # Imports
from scipy.io import wavfile
import scipy.signal as sps

# Your new sampling rate
new_rate = 16000

for filename in tqdm(glob.glob(os.path.join(path_Voice, '*.wav'))):
    sampling_rate, data = wavfile.read(filename)
    number_of_samples = round(len(data) * float(new_rate) / sampling_rate)
    data = sps.resample(data, number_of_samples)
    data = np.asarray(data, dtype=np.int16)
    wavfile.write(filename,new_rate,data)

In [ ]:
filename = glob.glob(os.path.join(path_Voice, '*.wav'))
print(filename)

In [ ]:
# tqdm(glob.glob(os.path.join(path_Voice, '*.wav')))

In [ ]:
for filename in tqdm(glob.glob(os.path.join(path_Voice, '*.wav'))):
    myaudio = AudioSegment.from_file(filename) 
    chunk_length_ms = 1000 # pydub calculates in millisec
    chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec
    
    for i, chunk in enumerate(chunks):
        
        chunk_name = "chunk{0}.wav".format(i)
        name = path_Voice+'/'+chunk_name
        
        chunk.export(name, format="wav")
        
        rate, data = wavfile.read(path_Voice+'/'+chunk_name)
        
        
        if len(data.shape) >= 2 and data.size > 0:
            if data.shape[-1] > 1:
                data = data.mean(axis=-1)
            else:
                data = np.reshape(data, data.shape[:-1])
        img = generateSpectrogramForWave(data)
        np.save(path_Voice+'/'+chunk_name[:-4]+'.npy', img) # save

In [ ]:
import tensorflow as tf
from tensorflow import keras
new_model = tf.keras.models.load_model('/kaggle/input/modelh5/model.h5')

# Check its architecture
new_model.summary()

In [ ]:
(glob.glob(os.path.join(path_Voice, '*.npy')))

In [ ]:
(glob.glob(os.path.join(path_Voice, '*.npy')))
img_test = np.load('/kaggle/working/samples/chunk0.npy')

In [ ]:
img_test.shape

In [ ]:
# test_pred

In [ ]:
ROW = 257
COL = 62

test_pred = []
count = 0
rate = 16000

for filename in tqdm(glob.glob(os.path.join(path_Voice, '*.npy'))):
    img_test = np.load(filename)
    row_,col_,_ = img_test.shape
    
    if col_ <COL:
        continue
    
    print(img_test.shape)
    
    img_test = img_test/255
    img_test = img_test.reshape(-1, ROW,COL,3)
    decoded_imgs = new_model.predict(img_test) #predict
    decoded_imgs = decoded_imgs.reshape(ROW,COL,3)
    decoded_imgs = decoded_imgs*255
    decoded_imgs = decoded_imgs.astype(np.int16)
    data = recoverSignalFromSpectrogram(decoded_imgs)
    file = './'+"testpred_{}".format(count)+'.wav'
    scipy.io.wavfile.write(file, rate, data)
    test_pred.append(file)
    count = count+1

In [ ]:
test_pred

In [ ]:
from pydub import AudioSegment
sound = 0
path_recovered = '/kaggle/working/noisesB/samples'
for i in range(len(test_pred)):
    print(test_pred[i])
    sound += AudioSegment.from_wav(test_pred[i])
    os.remove(test_pred[i])
sound.export(path_recovered+"soundJoined.wav", format="wav")

In [ ]:
sound.export("/kaggle/working/samples/"+"soundJoined.wav", format="wav")

In [ ]:
def signaltonoise(a, axis, ddof): 
    a = np.asanyarray(a) 
    m = a.mean(axis) 
    sd = a.std(axis = axis, ddof = ddof) 
    return np.where(sd == 0, 0, m / sd)
import scipy.io.wavfile as wavfile
import numpy
import os.path
from scipy import stats


# def signaltonoise(a, axis=0, ddof=0):
#     a = np.asanyarray(a)
#     m = a.mean(axis)
#     sd = a.std(axis=axis, ddof=ddof)
#     return np.where(sd == 0, 0, m/sd)

# def signaltonoise(Arr, axis=0, ddof=0):
#     Arr = np.asanyarray(Arr)
#     me = Arr.mean(axis)
#     sd = Arr.std(axis=axis, ddof=ddof)
#     return np.where(sd == 0, 0, me/sd)

    
def signaltonoise(a, axis=0, ddof=0):
    mx = np.amax(a)
    a = np.divide(a,mx)
    a = np.square(a)
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=ddof)
    return np.where(sd == 0, 0, m/sd)


# def snr(file):
#   if (os.path.isfile(file)):
#     data = wavfile.read(file)[1]
#     singleChannel = data
#     try:
#       singleChannel = numpy.sum(data, axis=1)
#     except:
#       # was mono after all
#       pass
      
#     norm = singleChannel / (max(numpy.amax(singleChannel), -1 * numpy.amin(singleChannel)))
#     return stats.signaltonoise(norm)

In [ ]:
import IPython.display as ipd
# ipd.Audio("/kaggle/working/samples/196-122150-0032.wav")

In [ ]:
clean = '/kaggle/working/noisesB/results'
noisy = '/kaggle/working/noisesB/samples'

In [ ]:
for filename in tqdm(glob.glob(os.path.join(noisy, '*.wav'))):
    data = wavfile.read(filename)[1]
    
    #Single Channel
    if len(data.shape) >= 2 and data.size > 0:
        if data.shape[-1] > 1:
            data = data.mean(axis=-1)
        else:
            data = np.reshape(data, data.shape[:-1])
    snr = signaltonoise(data)
    snr_db = 10 * np.log10(snr) 
    print(filename,snr,snr_db)

In [ ]:
for filename in tqdm(glob.glob(os.path.join(clean, '*.wav'))):
    data = wavfile.read(filename)[1]
    
    #Single Channel
    if len(data.shape) >= 2 and data.size > 0:
        if data.shape[-1] > 1:
            data = data.mean(axis=-1)
        else:
            data = np.reshape(data, data.shape[:-1])
            
    snr = signaltonoise(data)
    snr_db = 10 * np.log10(snr) 
    print(filename,snr,snr_db)

In [ ]:
import os
from tqdm import tqdm
import glob
import shutil

for filename in glob.glob(os.path.join(clean, '*.png')):
    os.remove(filename)
for filename in glob.glob(os.path.join(noisy, '*.png')):
    os.remove(filename)

In [ ]:
glob.glob(os.path.join("/kaggle/working/results", '*.png'))

In [ ]:
wav_list = os.listdir(clean)
num_wavs = len(clean)

In [ ]:
os.listdir(clean)

In [ ]:
wav_list

In [ ]:
for n in range(num_wavs-2):
    name_A = wav_list[n]
    path_A = os.path.join(clean,name_A)
    rate, data = wavfile.read(path_A)
    # Average over windows.
    if len(data.shape) >= 2 and data.size > 0:
        if data.shape[-1] > 1:
            data = data.mean(axis=-1)
        else:
            data = np.reshape(data, data.shape[:-1])
    # Only use a short clip for our demo
    if np.shape(data)[0] / float(rate) > 5:
        data = data[0 : rate * 5]
    img = generateSpectrogramForWave(data)
    img.save(path_A[:-4]+'.png')

In [ ]:
  img.save(path_A[:-4]+'.png')

In [ ]:
#combined clean sound
ipd.Audio("/kaggle/working/noisesB/samplessoundJoined.wav")

In [ ]:
##clean
ipd.Audio("/kaggle/working/noisesB/results/amb.wav")

In [ ]:
##noisy
ipd.Audio("/kaggle/working/noisesB/samples/amb.wav")

In [ ]:
import librosa

In [ ]:
from IPython.display import Image
Image("/kaggle/working/noisesB/results/3982-182255-0009.png")

In [ ]:
Image("/kaggle/working/noisesB/results/amb.png")

In [ ]:
# testing files

In [ ]:
from tensorflow import keras

# Load the model
model = keras.models.load_model('/kaggle/working/model_test_run')

In [ ]:
# Access the weights of the model
model_weights = model.get_weights()

# Print the weights
for layer_weights in model_weights:
    print(layer_weights)

In [ ]:
with tpu_strategy.scope():
    load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
    model = tf.keras.models.load_model('./kaggle/working/model_test_run', options=load_locally) # loading in Tensorflow's "SavedModel" format

In [ ]:
import numpy as np
import math
from PIL import Image
import time

# Packages we're using
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage
from tqdm import tqdm

%matplotlib inline
import IPython.display
from ipywidgets import interact, interactive, fixed

# Packages we're using
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage

import glob
import os
from tqdm import tqdm
# plt, plot, tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import shutil

file_path = "/kaggle/input/noise-cancellation-dataset/NoisySpeech_training_singleNoise1/NoisySpeech_training_singleNoise1/noisy101_SNRdb_0.0_clnsp101.wav"
new_directory = "/kaggle/working/"

shutil.move(file_path, new_directory)

In [ ]:
!pip install pydub

In [ ]:
import shutil

source_file = '/kaggle/input/noise-cancellation-dataset/NoisySpeech_training/noisy100_SNRdb_0.0_clnsp100.wav'
destination_file = '/kaggle/working/noisy100_SNRdb_0.0_clnsp100.wav'

# Copy the file
shutil.copy(source_file, destination_file)

In [ ]:
import pydub
filename = "/kaggle/working/noisy100_SNRdb_0.0_clnsp100.wav"
import tensorflow as tf
from tensorflow import keras
model = tf.keras.models.load_model('/kaggle/working/model_sgd.h5')
import numpy as np
import math
FFT_LENGTH = 512
WINDOW_LENGTH = 512
WINDOW_STEP = int(WINDOW_LENGTH / 2)

phaseMax = 3.141592653589793 
phaseMin = -3.141592653589793
magnitudeMax = 2211683.973249525
magnitudeMin = 0.0
def amplifyMagnitudeByLog(d):
    return 188.301 * math.log10(d + 1)

def weakenAmplifiedMagnitude(d):
    return math.pow(10, d/188.301)-1

def generateLinearScale(magnitudePixels, phasePixels, 
                        magnitudeMin, magnitudeMax, phaseMin, phaseMax):
    height = magnitudePixels.shape[0]
    width = magnitudePixels.shape[1]
    magnitudeRange = magnitudeMax - magnitudeMin
    phaseRange = phaseMax - phaseMin
    rgbArray = np.zeros((height, width, 3), 'uint8')
    
    for w in range(width):
        for h in range(height):
            magnitudePixels[h,w] = (magnitudePixels[h,w] - magnitudeMin) / (magnitudeRange) * 255 * 2
            magnitudePixels[h,w] = amplifyMagnitudeByLog(magnitudePixels[h,w])
            phasePixels[h,w] = (phasePixels[h,w] - phaseMin) / (phaseRange) * 255
            red = 255 if magnitudePixels[h,w] > 255 else magnitudePixels[h,w]
            green = (magnitudePixels[h,w] - 255) if magnitudePixels[h,w] > 255 else 0
            blue = phasePixels[h,w]
            rgbArray[h,w,0] = int(red)
            rgbArray[h,w,1] = int(green)
            rgbArray[h,w,2] = int(blue)
    return rgbArray

def recoverLinearScale(rgbArray, magnitudeMin, magnitudeMax, 
                       phaseMin, phaseMax):
    width = rgbArray.shape[1]
    height = rgbArray.shape[0]
#     print(phaseMax,phaseMin)
    magnitudeVals = rgbArray[:,:,0].astype(float) + rgbArray[:,:,1].astype(float)
    phaseVals = rgbArray[:,:,2].astype(float)
    phaseRange = phaseMax - phaseMin
    magnitudeRange = magnitudeMax - magnitudeMin
    
#     print(magnitudeVals)
#     print(phaseVals)
#     print(phaseRange)
#     print(magnitudeRange)
    
    for w in range(width):
        for h in range(height):
            phaseVals[h,w] = (phaseVals[h,w] / 255 * phaseRange) + phaseMin
            magnitudeVals[h,w] = weakenAmplifiedMagnitude(magnitudeVals[h,w])
            magnitudeVals[h,w] = (magnitudeVals[h,w] / (255*2) * magnitudeRange) + magnitudeMin
    return magnitudeVals, phaseVals
from PIL import Image
def generateSpectrogramForWave(signal):
    start_time = time.time()
    global magnitudeMin
    global magnitudeMax
    global phaseMin
    global phaseMax
    buffer = np.zeros(int(signal.size + WINDOW_STEP - (signal.size % WINDOW_STEP)))
    buffer[0:len(signal)] = signal
    height = int(FFT_LENGTH / 2 + 1)
    width = int(len(buffer) / (WINDOW_STEP) - 1)
    magnitudePixels = np.zeros((height, width))
    phasePixels = np.zeros((height, width))

    for w in range(width):
        buff = np.zeros(FFT_LENGTH)
        stepBuff = buffer[w*WINDOW_STEP:w*WINDOW_STEP + WINDOW_LENGTH]
        # apply hanning window
        stepBuff = stepBuff * np.hanning(WINDOW_LENGTH)
        buff[0:len(stepBuff)] = stepBuff
        #buff now contains windowed signal with step length and padded with zeroes to the end
        fft = np.fft.rfft(buff)
        for h in range(len(fft)):
            #print(buff.shape)
            #return
            magnitude = math.sqrt(fft[h].real**2 + fft[h].imag**2)
            if magnitude > magnitudeMax:
                magnitudeMax = magnitude 
            if magnitude < magnitudeMin:
                magnitudeMin = magnitude 

            phase = math.atan2(fft[h].imag, fft[h].real)
            if phase > phaseMax:
                phaseMax = phase
            if phase < phaseMin:
                phaseMin = phase
            magnitudePixels[height-h-1,w] = magnitude
            phasePixels[height-h-1,w] = phase
    rgbArray = generateLinearScale(magnitudePixels, phasePixels,
                                  magnitudeMin, magnitudeMax, phaseMin, phaseMax)
    
    
    elapsed_time = time.time() - start_time
    print('%.2f' % elapsed_time, 's', sep='')
#     print(rgbArray.shape)
#     rgbArray=rgbArray.reshape((257,62,3))
    img = Image.fromarray(rgbArray, 'RGB')
    return img
def recoverSignalFromSpectrogram(numpyarray):
#     img = Image.open(filePath)
    data = np.array( numpyarray, dtype='uint8' )
    width = data.shape[1]
    height = data.shape[0]
    
#     print(phaseMax,phaseMin)

    magnitudeVals, phaseVals \
    = recoverLinearScale(data, magnitudeMin, magnitudeMax, phaseMin, phaseMax)
    
#     print(magnitudeVals,phaseVals)
    
    recovered = np.zeros(WINDOW_LENGTH * width // 2 + WINDOW_STEP, dtype=np.int16)
    recovered = np.array(recovered,dtype=np.int16)
    
    for w in range(width):
        toInverse = np.zeros(height, dtype=np.complex_)
        for h in range(height):
            magnitude = magnitudeVals[height-h-1,w]
            phase = phaseVals[height-h-1,w]
            toInverse[h] = magnitude * math.cos(phase) + (1j * magnitude * math.sin(phase))
        signal = np.fft.irfft(toInverse)
        recovered[w*WINDOW_STEP:w*WINDOW_STEP + WINDOW_LENGTH] += signal[:WINDOW_LENGTH].astype(np.int16)
    return recovered
rate = 16000


from scipy.io import wavfile
import scipy.signal as sps
sampling_rate, data = wavfile.read(filename)
number_of_samples = round(len(data) * float(rate) / sampling_rate)
data = sps.resample(data, number_of_samples)
data = np.asarray(data, dtype=np.int16)
wavfile.write(filename,rate,data)
filename1 = '/kaggle/working/'
from pydub import AudioSegment
from pydub.utils import make_chunks
import time


myaudio = AudioSegment.from_file(filename)
chunk_length_ms = 1000 # pydub calculates in millisec
chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec
    
for i, chunk in enumerate(chunks):
    print(i)
    print(chunk)
        
    chunk_name = "chunk1{0}.wav".format(i)
    name = filename1+'/'+chunk_name
    
    chunk.export(name, format="wav")
    
    rate, data = wavfile.read(name)
    
    
    if len(data.shape) >= 2 and data.size > 0:
        if data.shape[-1] > 1:
            data = data.mean(axis=-1)
        else:
            data = np.reshape(data, data.shape[:-1])
    img = generateSpectrogramForWave(data)
#     print(img)
#     img=img.reshape((257))
    np.save(filename1+'/'+chunk_name[:-4]+'.npy', img) # save

ROW = 257
COL = 62


# file_list = sorted_files  # Replace with your list of file names
# destination_folder = '/kaggle/working/chunks/'  # Replace with the path to the new folder

# # Create the destination folder if it doesn't exist
# if not os.path.exists(destination_folder):
#     os.makedirs(destination_folder)

# # Move files from the list to the destination folder
# for file_name in file_list:
#     source_path = file_name  # Assuming the files are in the current working directory
#     destination_path = os.path.join(destination_folder, file_name)
#     shutil.move(source_path, destination_path)
# directory = filename1  # Replace with the path to the directory containing the files

# # Get the list of files in the directory
# files = os.listdir(directory)

# # Sort the files based on their names
# sorted_files = sorted(files)
# my_list = sorted_files  # Example list
# current_index = [6,7,8,9]  # Index of the item to move

# # Remove the item from the current index
# i = 0
# while i<4:    
#     item = my_list.pop(6)
#     my_list.append(item)
#     i=i+1

# # Insert the item at the new index
# my_list

In [ ]:
test_pred = []
import glob
import os
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage
from tqdm import tqdm
count = 0
rate = 16000

# for filename in tqdm(glob.glob('path/to/files/*.npy')):
#     img_test = np.load(filename)
#     row_,col_,_ = img_test.shape
#     print(filename, col_)
    
for i, _ in enumerate(chunks):
    filename = '/kaggle/working/'+'chunk1{0}'.format(i)+'.npy'
    img_test = np.load(filename)
    row_,col_,_ = img_test.shape
    
    if col_ < COL:
        continue
    
    print(img_test.shape)
    
    img_test = img_test/255
    img_test = img_test.reshape(-1, ROW,COL,3)
    
    decoded_imgs = model.predict(img_test) #predict
    
    decoded_imgs = decoded_imgs.reshape(ROW,COL,3)
    decoded_imgs = decoded_imgs*255
    decoded_imgs = decoded_imgs.astype(np.int16)
    data = recoverSignalFromSpectrogram(decoded_imgs) # save predicted audio
    file = "/kaggle/working/" +"testpred_{}".format(count)+'.wav' #created file
    scipy.io.wavfile.write(file, rate, img_test) #wrote the file with 'data'
    test_pred.append(file) #saves file in array
    count = count+1

from pydub import AudioSegment

sound = 0
path_recovered = '/kaggle/working/recovered/'
if not os.path.exists(path_recovered):
    os.makedirs(path_recovered)

In [ ]:
AudioSegment.from_wav(test_pred[0],)

In [ ]:
for i in range(len(test_pred)):
    print(test_pred[i])
    sound += AudioSegment.from_wav(test_pred[i])
#     os.remove(test_pred[i])
sound.export(path_recovered+"soundJoined1.wav", format="wav")

In [ ]:
test_pred

In [ ]:
from pydub import AudioSegment
sound = 0
path_recovered = '/kaggle/working/reco'
for i in range(len(test_pred)):
    print(test_pred[i])
    sound += AudioSegment.from_wav(test_pred[i])
    os.remove(test_pred[i])
sound.export(path_recovered+"soundJoined.wav", format="wav")

In [ ]:
import IPython.display as ipd
ipd.Audio("/kaggle/working/recosoundJoined.wav")

In [ ]:
import os
import soundfile as sf
import numpy as np

path_recovered = '/kaggle/working/reco'
output_file = path_recovered + "soundJoined.wav"

# Initialize variables
sound = []
sample_rate = 16000

for i in range(len(test_pred)):
    print(test_pred[i])
    audio, sr = sf.read(test_pred[i])
    
    if sample_rate is None:
        sample_rate = sr
    
    # Resample audio if necessary
    if sr != sample_rate:
        audio = np.interp(
            np.arange(0, len(audio), sr / sample_rate),
            np.arange(0, len(audio)),
            audio
        )
    
    sound.append(audio)

    # Remove the file
    os.remove(test_pred[i])

# Ensure consistent dimensions
max_length = max(len(audio) for audio in sound)
sound = [np.pad(audio, (0, max_length - len(audio)), mode='constant') for audio in sound]

# Concatenate the audio segments
sound = np.concatenate(sound)

# Save the joined audio
sf.write(output_file, sound, sample_rate, 'PCM_16')

print("Joined audio saved to:", output_file)